In [24]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from keras.preprocessing.text import Tokenizer
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import re

In [36]:
dataframe = pd.read_csv('sentiment_data/Sentiment.csv')
dataframe = dataframe[['text','sentiment']]
dataframe = dataframe[dataframe.sentiment != 'Neutral']
dataframe['text'] = dataframe['text'].apply(lambda x:x.lower())
dataframe['text'] = dataframe['text'].apply(lambda x:re.sub('[^a-zA-z0-9\s]','',x))
for idx,row in dataframe.iterrows():
    row[0] = row[0].replace('rt',' ')

nb_size = 2000
tokenizer = Tokenizer(num_words = nb_size , split=' ')
tokenizer.fit_on_texts(dataframe['text'].values)
X = tokenizer.texts_to_sequences(dataframe['text'].values)
X = pad_sequences(X)
#print X[0:5]
#dataframe.head()

In [44]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(nb_size,embed_dim,input_length = X.shape[1],dropout = 0.2))
model.add(LSTM(lstm_out,dropout_U=0.2,dropout_W=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
print model.summary()

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:5: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:6: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(196, dropout=0.2, recurrent_dropout=0.2)`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 28, 128)           256000    
_________________________________________________________________
lstm_3 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [45]:
Y = pd.get_dummies(dataframe['sentiment'].values)
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)
print X_train[0]

((7188, 28), (7188, 2))
((3541, 28), (3541, 2))
[  0   0   0   0   0   0   0   0  72  11  80 312   7 980 781 758 101  28
  99 234 278  21 375 199 414   6   7 185]


In [46]:
batch_size = 32
X_train = np.array(X_train)
Y_train = np.array(Y_train)
model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size, verbose = 2)

Epoch 1/7
38s - loss: 0.4292 - acc: 0.8203
Epoch 2/7
44s - loss: 0.3083 - acc: 0.8702
Epoch 3/7
46s - loss: 0.2655 - acc: 0.8881
Epoch 4/7
44s - loss: 0.2330 - acc: 0.9035
Epoch 5/7
44s - loss: 0.2015 - acc: 0.9168
Epoch 6/7
40s - loss: 0.1825 - acc: 0.9236
Epoch 7/7
37s - loss: 0.1562 - acc: 0.9373


In [47]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = np.array(X_test)
Y_test = np.array(Y_test)
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

score: 0.52
acc: 0.83
